## Creating the OD Buffer and Network buffers from the flood map extent

### OD Buffer, Network Buffer

In [2]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box
import rasterio
import matplotlib.pyplot as plt

In [3]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper,)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter


In [9]:
# specify the name of the path to the project folder where you created the RA2CE folder setup

root_dir = Path(r'P:\moonshot2-casestudy\RA2CE_HACKATHON_JULY\Automating_Destinations_and_buffer_creation')
assert root_dir.exists()

static_path = root_dir.joinpath("static")
hazard_path = static_path.joinpath("hazard")
network_path = static_path.joinpath("network")

### Determine the extent of the flood maps to download a road network and Origins and Destinations and add 5% to get the OD buffer and 10% to get the network buffer

In [6]:
import os
import rasterio


# Initialize an empty list to store GeoTIFF file names
geotiff_files = []


# Find all GeoTIFF files in the folder
for filename in os.listdir(hazard_path):
    if filename.lower().endswith((".tif", ".tiff")):
        geotiff_files.append(os.path.join(hazard_path, filename))

# Initialize variables for overall extent
min_lon, min_lat = float("inf"), float("inf")
max_lon, max_lat = float("-inf"), float("-inf")

# Loop over the GeoTIFF files
for geotiff_file in geotiff_files:
    with rasterio.open(geotiff_file) as src:
        #print(src.crs)
        bounds = src.bounds
       # print(geotiff_files,src.crs)
        
        min_lon = min(min_lon, bounds.left)
        min_lat = min(min_lat, bounds.bottom)
        max_lon = max(max_lon, bounds.right)
        max_lat = max(max_lat, bounds.top)

# Overall geospatial extent
#print(f"Min Longitude: {min_lon}, Max Longitude: {max_lon}")
#print(f"Min Latitude: {min_lat}, Max Latitude: {max_lat}")

# Create a Shapely polygon
polygon = box(min_lon, min_lat, max_lon, max_lat)

gdf_polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])

# Calculate width and height
height = max_lon - min_lon
width = max_lat - min_lat
height_OD_buffer = height*0.05
height_network_buffer = height*0.1

buffer_polygon_OD = polygon.buffer(height_OD_buffer, quad_segs=4,cap_style='square')
buffer_polygon_network = polygon.buffer(height_network_buffer)

#print(buffer_polygon_OD)

In [7]:
#If you wanna visualize
gdf_buffer_polygon_OD = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[buffer_polygon_OD])
gdf_buffer_polygon_network = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[buffer_polygon_network])

import folium
m = gdf_polygon.explore(color='red')
m = gdf_buffer_polygon_OD.explore(m=m, color='blue')
m = gdf_buffer_polygon_network.explore(m=m, color='green')
folium.LayerControl().add_to(m)
m

In [10]:
#create Json files of polygons
#gdf_buffer_polygon_OD.to_file('P:\moonshot2-casestudy\RA2CE_HACKATHON_JULY\Automating_Destinations_and_buffer_creation\buffer_polygon_OD.geojson', driver='GeoJSON')

import json
from shapely.geometry import mapping

def to_geojson(geometry, file_path, indent=None, **kwargs):
    geojson = mapping(geometry)
    with open(file_path, 'w') as f:
        json.dump(geojson, f, indent=indent, **kwargs)


to_geojson(gdf_buffer_polygon_network, network_path/'buffer_polygon_network.geojson')
to_geojson(buffer_polygon_OD, network_path/'buffer_polygon_OD.geojson')